In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import pprint
import re
import requests as req

client = MongoClient('mongodb://localhost:27017/')
db = client.companies

PyMongo query to keep only companies with office longitude and latitude information available:

In [3]:
query={'$and': [{"offices.latitude":{'$not':{'$eq':None}}},{"offices.longitude":{'$not':{'$eq':None}}},{"offices.latitude":{'$exists':True}},{"offices.longitude":{'$exists':True}}]}
projection={'_id':0,'category_code':1,'name':1,'number_of_employees':1,'founded_year':1,'deadpooled_year':1,'tag_list':1,'description':1,'total_money_raised':1,"offices.latitude":1,"offices.longitude":1}
with_geoloc=db.companies.find(query,projection)

Make a dataframe from the previous query:

In [4]:
df=pd.DataFrame(with_geoloc)


Re-order columns:

In [5]:
df=df[['name','category_code','description','founded_year','deadpooled_year','number_of_employees','total_money_raised','tag_list','offices']]
df.head(3)

,name,category_code,description,founded_year,deadpooled_year,number_of_employees,total_money_raised,tag_list,offices
0,AdventNet,enterprise,Server Management Software,1996.0,2.0,600.0,$0,,"[{'latitude': 37.692934, 'longitude': -121.904..."
1,Wetpaint,web,Technology Platform Company,2005.0,1.0,47.0,$39.8M,"wiki, seattle, elowitz, media-industry, media-...","[{'latitude': 47.603122, 'longitude': -122.333..."
2,Zoho,software,Online Business Apps Suite,2005.0,3.0,1600.0,$0,"zoho, officesuite, spreadsheet, writer, projec...","[{'latitude': 37.692934, 'longitude': -121.904..."


Get exchange rate from API to "clean" currency:

In [6]:
url="https://api.exchangeratesapi.io/latest"
query_params={"base":"USD","symbols":["CAD","EUR","GBP","SEK"]}

res = req.get(url,params=query_params)
content=res.json()
rates=content["rates"]

rates

{'CAD': 1.3124497992,
 'EUR': 0.8924587238,
 'GBP': 0.8029897367,
 'SEK': 9.490227577}

In [7]:
def money_dollars(s):
    mfloat=float(re.search("(\d*\.\d*|\d+)",s).group(1))
    if s.endswith("M"): mfloat*=1000000
    if s.endswith("B"): mfloat*=1000000000
    if s.endswith("k"): mfloat*=1000
    if s.startswith("C$"): mfloat*=1/rates["CAD"]
    if s.startswith("€"): mfloat*=1/rates["EUR"]
    if s.startswith("£"): mfloat*=1/rates["GBP"]
    if s.startswith("kr"): mfloat*=1/rates["SEK"]
    return round(mfloat)

df["raised_money_dollars"]=df.copy()["total_money_raised"].apply(money_dollars)
df=df[['name','category_code','description','founded_year','deadpooled_year','number_of_employees','total_money_raised',"raised_money_dollars",'tag_list','offices']]
df.reset_index(inplace=True)
df.head(3)

,index,name,category_code,description,founded_year,deadpooled_year,number_of_employees,total_money_raised,raised_money_dollars,tag_list,offices
0,0,AdventNet,enterprise,Server Management Software,1996.0,2.0,600.0,$0,0,,"[{'latitude': 37.692934, 'longitude': -121.904..."
1,1,Wetpaint,web,Technology Platform Company,2005.0,1.0,47.0,$39.8M,39800000,"wiki, seattle, elowitz, media-industry, media-...","[{'latitude': 47.603122, 'longitude': -122.333..."
2,2,Zoho,software,Online Business Apps Suite,2005.0,3.0,1600.0,$0,0,"zoho, officesuite, spreadsheet, writer, projec...","[{'latitude': 37.692934, 'longitude': -121.904..."


In [35]:
offices_list=df.copy()[["index","offices"]]
offices_list_clean= pd.DataFrame(offices_list["offices"].tolist()).stack().reset_index(level=1, drop=True).reset_index(name='offices')

display(offices_list_clean.head(5))

,index,offices
0,0,"{'latitude': 37.692934, 'longitude': -121.904945}"
1,1,"{'latitude': 47.603122, 'longitude': -122.333253}"
2,1,"{'latitude': 40.7237306, 'longitude': -73.9964..."
3,2,"{'latitude': 37.692934, 'longitude': -121.904945}"
4,3,"{'latitude': 37.764726, 'longitude': -122.394523}"


In [9]:
df_offices=df.merge(offices_list_clean,left_on='index', right_on='index')
df_offices.head(3)

,index,name,category_code,description,founded_year,deadpooled_year,number_of_employees,total_money_raised,raised_money_dollars,tag_list,offices_x,offices_y
0,0,AdventNet,enterprise,Server Management Software,1996.0,2.0,600.0,$0,0,,"[{'latitude': 37.692934, 'longitude': -121.904...","{'latitude': 37.692934, 'longitude': -121.904945}"
1,1,Wetpaint,web,Technology Platform Company,2005.0,1.0,47.0,$39.8M,39800000,"wiki, seattle, elowitz, media-industry, media-...","[{'latitude': 47.603122, 'longitude': -122.333...","{'latitude': 47.603122, 'longitude': -122.333253}"
2,1,Wetpaint,web,Technology Platform Company,2005.0,1.0,47.0,$39.8M,39800000,"wiki, seattle, elowitz, media-industry, media-...","[{'latitude': 47.603122, 'longitude': -122.333...","{'latitude': 40.7237306, 'longitude': -73.9964..."


In [10]:
df_offices = df_offices.drop(["offices_x"], axis=1)
df_offices.rename(index=str, columns={"offices_y": "offices"},inplace=True)
df_offices.head(3)

,index,name,category_code,description,founded_year,deadpooled_year,number_of_employees,total_money_raised,raised_money_dollars,tag_list,offices
0,0,AdventNet,enterprise,Server Management Software,1996.0,2.0,600.0,$0,0,,"{'latitude': 37.692934, 'longitude': -121.904945}"
1,1,Wetpaint,web,Technology Platform Company,2005.0,1.0,47.0,$39.8M,39800000,"wiki, seattle, elowitz, media-industry, media-...","{'latitude': 47.603122, 'longitude': -122.333253}"
2,1,Wetpaint,web,Technology Platform Company,2005.0,1.0,47.0,$39.8M,39800000,"wiki, seattle, elowitz, media-industry, media-...","{'latitude': 40.7237306, 'longitude': -73.9964..."


In [18]:
def get_coords(data):
    data = data['offices']
#    return (len(data),data[0]['latitude'],data[0]['longitude'])

    # Only create the geoJSON object if all geodata is available
    principal = None
    if data['latitude'] and data['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data['longitude'], data['latitude']]
        }

    return {
        "lat": data['latitude'],
        "lng": data['longitude'],
        "geo_point": principal
    }


offices = df_offices[["offices"]].apply(get_coords, result_type="expand", axis=1)

In [21]:
df_geo=pd.concat([df_offices,offices], axis=1)
df_geo = df_geo.drop(["offices"], axis=1)

In [29]:
df_geo["category_code"]=df_geo["category_code"].fillna("None")
df_geo["description"]=df_geo["description"].fillna("None")
df_geo["description"]=df_geo["description"].fillna("None")
df_geo["tag_list"]=df_geo["tag_list"].fillna("None")
df_geo["founded_year"]=df_geo["founded_year"].fillna(0)
df_geo["deadpooled_year"]=df_geo["deadpooled_year"].fillna(0)
df_geo["number_of_employees"]=df_geo["number_of_employees"].fillna(0)
df_geo.isnull().sum()

index                   0
name                    0
category_code           0
description             0
founded_year            0
deadpooled_year         0
number_of_employees     0
total_money_raised      0
raised_money_dollars    0
tag_list                0
geo_point               0
lat                     0
lng                     0
dtype: int64

In [31]:
df_geo["founded_year"]=df_geo["founded_year"].astype('int')
df_geo["deadpooled_year"]=df_geo["deadpooled_year"].astype('int')
df_geo["number_of_employees"]=df_geo["number_of_employees"].astype('int')
df_geo.dtypes

index                     int64
name                     object
category_code            object
description              object
founded_year              int64
deadpooled_year           int64
number_of_employees       int64
total_money_raised       object
raised_money_dollars      int64
tag_list                 object
geo_point                object
lat                     float64
lng                     float64
dtype: object

In [33]:
df_geo = df_geo.drop(["total_money_raised"], axis=1)

In [36]:
df_geo.to_json('../data/df_geo.json', orient="records")